# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff
using PseudoPotentialData

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice = T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    pseudopotentials = PseudoFamily("cp2k.nc.sr.lda.v0_1.semicore.gth")
    atoms     = [ElementPsp(:He, pseudopotentials)]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions;
                      functionals=[:lda_x, :lda_c_vwn],
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770882826177                   -0.52    9.0    173ms
  2   -2.772149004635       -2.90       -1.32    1.0    128ms
  3   -2.772170297265       -4.67       -2.44    1.0    121ms
  4   -2.772170652540       -6.45       -3.15    1.0    104ms
  5   -2.772170722903       -7.15       -4.42    2.0    125ms
  6   -2.772170722961      -10.24       -4.54    1.0    111ms
  7   -2.772170723012      -10.29       -5.43    1.0    117ms
  8   -2.772170723014      -11.72       -5.64    1.0    120ms
  9   -2.772170723015      -12.14       -6.57    2.0    136ms
 10   -2.772170723015      -13.78       -6.84    1.0    119ms
 11   -2.772170723015      -13.78       -7.38    1.0    121ms
 12   -2.772170723015      -14.88       -7.75    1.0    131ms
 13   -2.772170723015   +  -13.97       -8.55    1.0    1.09s
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ----

1.7735580114553477

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem. This leads to a density-functional perturbation
theory problem, which is automatically set up and solved in the background.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770775926047                   -0.52    8.0    193ms
  2   -2.772056940426       -2.89       -1.32    1.0    101ms
  3   -2.772083057659       -4.58       -2.51    1.0    119ms
  4   -2.772083362439       -6.52       -3.31    1.0    107ms
  5   -2.772083415633       -7.27       -3.85    2.0    128ms
  6   -2.772083417738       -8.68       -5.08    1.0    111ms
  7   -2.772083417810      -10.14       -5.62    2.0    134ms
  8   -2.772083417811      -12.88       -6.10    1.0    117ms
  9   -2.772083417811      -13.45       -6.98    1.0    119ms
 10   -2.772083417811      -13.70       -7.57    1.0    119ms
 11   -2.772083417811   +  -14.05       -8.04    2.0    134ms
Solving response problem
Iter  Restart  Krydim  log10(res)  avg(CG)  Δtime   Comment
----  -------  ------  ----------  -------  ------  ---------------
                                      13.0